In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import SimpleITK as sitk
import os as os
import nrrd as reader

In [ ]:
def check_path(path):
    count=0
    for files in os.listdir(path):
        image_path = os.path.join(path,files)
        count+=1
    return count - 1

In [ ]:
# (patient count* 15) check_path function will return total patient count 
folder_path = os.getcwd() + "/Incoming Annotations"

length = check_path(folder_path)
inputs = np.zeros((length*15,256,512, 1))
outputs = np.zeros((length*15,256,512 , 9))

In [ ]:
def dcm_to_np(path):
    global dic_count
    try:
        slice_filenames = sitk.ImageSeriesReader_GetGDCMSeriesFileNames(path)
        image = sitk.ReadImage(slice_filenames)
        for x in range(15):
            current_slice = image[:, :, x]
            current_array = sitk.GetArrayFromImage(current_slice)
            inputs[dic_count]=current_array[:,:,np.newaxis]
            dic_count+=1
        print(dic_count)
    except Exception as e:
        print("Problem with this folder -> " + path)

In [ ]:
def nrrd_to_np(path):
    global ann_count
    try:
#         seg = sitk.GetArrayFromImage(meta_object)
        data, header = reader.read(path)
        data = data.transpose((1, 0, 2))
        for p in range(15):
            current_seg = data[p, :, :]
            new_seg = np.zeros([256,512,9])
            for y, a in enumerate(current_seg):
                for x, b in enumerate(a):
                    if b>0:
                        new_seg[y][x][b-1] = 1
            outputs[ann_count]=new_seg
            ann_count+=1
        print(ann_count)
    except Exception as e:
        print(e)

In [ ]:
def join(one, two):
    return os.path.join(one, two)

In [ ]:
dic_count = 0
ann_count = 0

try:
    for patient in sorted(os.listdir(folder_path)):
            patient_path = join(folder_path, patient)
            nrrd_folder = join(patient_path, "NRRD")
            dcm_to_np(patient_path)
            for nrrd in os.listdir(nrrd_folder):
                if(".nrrd" in nrrd):
                    nrrd_to_np(join(nrrd_folder, nrrd))
            break
    #                 segmentation = sitk.ReadImage(join(nrrd_folder, nrrd))
    #                 nrrd_to_np(segmentation)
except Exception as e:
    print(e)

In [ ]:
plt.imshow(inputs[0, :, :, 0])
plt.show()
plt.imshow(outputs[0, :, :, 6])
plt.show()

In [ ]:
# a = np.hstack(outputs[0].flatten())
# plt.hist(a, bins=2)
# plt.show()
unique, counts = np.unique(inputs[0], return_counts=True)
print(np.amax(inputs[0]))
plt.scatter(unique, counts)
plt.show()

In [ ]:
import sklearn.preprocessing as sk
scaler = sk.MinMaxScaler(feature_range=(0, 255))
inputs = scaler.fit_transform(inputs.flatten().reshape(-1, 1))

In [ ]:
for i in range(1740):
    plt.imshow(inputs[i].reshape(256, 512), cmap=plt.cm.bone)
    plt.imshow(outputs[i][:, :, 8], alpha=0.5)
    plt.show()

In [ ]:
from tensorflow.keras.models import *
from tensorflow.keras.layers import *

def unet_model(im_height, im_width, im_chan):

    input_img = Input((im_height, im_width, im_chan,), name='img')

    inp = BatchNormalization()(input_img)

    c1 = Conv2D(4, (3, 3), activation='relu', padding='same') (inp)
    a1 = MaxPooling2D((2, 2))(c1)
    c1 = Dropout(0.2)(c1)
    c1 = Conv2D(4, (3, 3), activation='relu', padding='same') (c1)
    p1 = MaxPooling2D((2, 2)) (c1)

    cat1 = concatenate([p1, a1])

    c2 = Conv2D(8, (3, 3), activation='relu', padding='same') (cat1)
    a2 = MaxPooling2D((2, 2))(c2)
    c2 = Dropout(0.2)(c2)
    c2 = Conv2D(8, (3, 3), activation='relu', padding='same') (c2)
    p2 = MaxPooling2D((2, 2)) (c2)

    cat2 = concatenate([p2, a2])

    c3 = Conv2D(16, (3, 3), activation='relu', padding='same') (cat2)
    a3 = MaxPooling2D((2, 2))(c3)
    c3 = Dropout(0.2)(c3)
    c3 = Conv2D(16, (3, 3), activation='relu', padding='same') (c3)
    p3 = MaxPooling2D((2, 2)) (c3)

    cat3 = concatenate([p3, a3])

    c4 = Conv2D(32, (3, 3), activation='relu', padding='same') (cat3)
    a4 = MaxPooling2D((2, 2))(c4)
    c4 = Dropout(0.2)(c4)
    c4 = Conv2D(32, (3, 3), activation='relu', padding='same') (c4)
    p4 = MaxPooling2D((2, 2)) (c4)

    cat4 = concatenate([p4, a4])

    c5 = Conv2D(64, (3, 3), activation='relu', padding='same') (cat4)
    a5 = MaxPooling2D((2, 2))(c5)
    c5 = Dropout(0.2)(c5)
    c5 = Conv2D(64, (3, 3), activation='relu', padding='same') (c5)
    p5 = MaxPooling2D((2, 2)) (c5)

    cat5 = concatenate([p5, a5])

    c6 = Conv2D(128, (3, 3), activation='relu', padding='same') (cat5)
    c6 = Dropout(0.2)(c6)
    c6 = Conv2D(128, (3, 3), activation='relu', padding='same') (c6)

    u7 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same') (c6)
    u7 = concatenate([u7, c5])
    c7 = Conv2D(64, (3, 3), activation='relu', padding='same') (u7)
    c7 = Dropout(0.2)(c7)
    c7 = Conv2D(64, (3, 3), activation='relu', padding='same') (c7)

    u8 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same') (c7)
    u8 = concatenate([u8, c4])
    c8 = Conv2D(32, (3, 3), activation='relu', padding='same') (u8)
    c8 = Dropout(0.2)(c8)
    c8 = Conv2D(32, (3, 3), activation='relu', padding='same') (c8)

    u9 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same') (c8)
    u9 = concatenate([u9, c3])
    c9 = Conv2D(16, (3, 3), activation='relu', padding='same') (u9)
    c9 = Dropout(0.2)(c9)
    c9 = Conv2D(16, (3, 3), activation='relu', padding='same') (c9)

    u10 = Conv2DTranspose(8, (2, 2), strides=(2, 2), padding='same') (c9)
    u10 = concatenate([u10, c2])
    c10 = Conv2D(8, (3, 3), activation='relu', padding='same') (u10)
    c10 = Dropout(0.2)(c10)
    c10 = Conv2D(8, (3, 3), activation='relu', padding='same') (c10)

    u11 = Conv2DTranspose(4, (2, 2), strides=(2, 2), padding='same') (c10)
    u11 = concatenate([u11, c1], axis = 3)
    c11 = Conv2D(4, (3, 3), activation='relu', padding='same') (u11)
    c11 = Dropout(0.2)(c11)
    c11 = Conv2D(4, (3, 3), activation='relu', padding='same') (c11)

    outputs = Conv2D(9, (1, 1), activation='sigmoid') (c11)

    model = Model(inputs=[input_img], outputs=[outputs])

    return model
model = unet_model(256, 512, 1)
model.summary()

In [ ]:
from sklearn.cross_validation import train_test_split

model.compile(optimizer='adam', loss="categorical_crossentropy", metrics=["binary_crossentropy"])
results = model.fit(x=inputs, y=outputs, epochs=60, batch_size=32)

In [ ]:
plt.title("Metrics")
plt.plot(results.history["loss"], label="loss")
plt.xlabel("Epochs")
plt.legend()

In [ ]:
result = model.predict(inputs[100].reshape(1, 256, 512, 1))
plt.imshow(inputs[100].reshape(256, 512), cmap=plt.cm.bone)
plt.show()
plt.imshow(result[0][:, :, 3])
plt.show()